# 泰坦尼克号生存预测（下）
在上节课中，我们通过泰坦尼克生存预测这个案例，已经基本了解了整个建模的流程步骤，同时也肯定对很多新出现的名词有些疑惑；今天这节课，我们会聚焦到流程中每个重要的知识点，以此为线索来理解机器学习技术背后的运作原理。

我们将一个个来学习以下知识点：
* 梯度下降法
* 学习速率
* 偏差与方差
* 过拟合与欠拟合
* 正则化
* 决策树原理
* bagging & boosting模型融合
* 分类模型评价指标：auc

## 1. 梯度下降法
在我们最开始讲解线性回归模型的时候，我们讲到线性回归模型的数学表达式是这个样子的：$$y = w_1x_1 + w_2x_2 +... + w_nx_n + b$$

而我们训练模型的过程其实就是找到最优的一组参数（$w_1, w_2...w_n, b$）使得最终的损失函数最小
$$MSE = \frac{1}{N} \sum_{(x,y)\in D} (y - prediction(x))^2$$
<img src="./material/训练试错.svg" width="700px" height="700px"/>
当时我们也说过，机器寻找参数的过程不是胡乱尝试的，而“梯度下降法”就是这个“计算参数更新”的算法。

先拿一个参数$w_1$来看，假设我们有时间和计算资源来计算$w_1$的所有可能值的损失。对于我们一直在研究的回归问题，所产生的损失与$w_1$的图形始终是凸形。换言之，图形始终是碗状图，如下所示：
### 回归问题产生的损失与权重图为凸形
<img src="./material/损失与权重凸形.svg" width="500px" height="500px"/>

凸形问题只有一个最低点；即只存在一个斜率正好为 0 的位置。这个最小值就是损失函数收敛之处。
梯度下降法的第一个阶段是为$w_1$选择一个起始值（起点）。起点并不重要；因此很多算法就直接将$w_1$设为 0 或随机选择一个值。下图显示的是我们选择了一个稍大于 0 的起点：
### 梯度下降起点
<img src="./material/权重起点.svg" width="500px" height="500px"/>
然后，梯度下降法算法会计算损失曲线在起点处的梯度。函数的梯度是偏导数相对于所有自变量的矢量，指向函数增长速度最快的方向。
对于单个权重的函数来说，梯度就等于导数。而梯度下降法算法则会沿着负梯度的方向走一步，以便尽快降低损失。

### 一个梯度步长将我们移动到损失曲线的下一个点
<img src="./material/梯度移动.svg" width="500px" height="500px"/>
梯度下降法会重复此过程，逐渐接近最低点。

我们用数学的方法来简单求证下这个过程，对损失函数：$$f(x)$$
则当x移动Δx,根据微分原理：$$f(x+Δx) = f(x) + f'(x)Δx + o(Δx)$$
这里o(Δx)为高阶无穷小，我们可以忽略，则公式变为$$f(x+Δx) = f(x) + f'(x)Δx$$
此时如果我们使Δx=-αf'(x)，也就是延损失函数负梯度的方向移动一点则:$$f(x+Δx) = f(x) + f'(x)[-αf'(x)]$$
即：$$f(x+Δx) = f(x) -α[f'(x)]^2$$
就可以得到此时：$$f(x+Δx) < f(x)$$
也就是说,沿着损失函数负梯度方向移动一步，损失函数变小了。
#### 结论
* 沿着损失函数负梯度的方向，损失函数会不断减小，而参数的更新就是损失函数不断计算梯度，损失下降的过程。
* 而移动的这个步长α就是**学习速率**。

## 2. 学习速率
梯度下降法算法用梯度乘以一个称为学习速率（有时也称为步长）的标量，以确定下一个点的位置。
学习速率的大小，会对我们的训练时间以及最终的模型结果产生影响。在大型的数据模型训练中，有可能一次训练就是几天甚至几周，因此速率的选择也是很重要的，毕竟青春只有一次！时间比金钱还要重要！

#### 速率过小
如果您选择的学习速率过小，就会花费太长的学习时间；
<img src="./material/速率过小.svg" width="500px" height="500px"/>

#### 速率过大
相反，如果您指定的学习速率过大，下一个点将永远在 U 形曲线的底部随意弹跳，就好像量子力学实验出现了严重错误一样，永远也不知道最好的点位置在哪里；
<img src="./material/速率过大.svg" width="500px" height="500px"/>

#### 速率合适
<img src="./material/速率合适.svg" width="500px" height="500px"/>
一般情况下，我们会先使用较大速率，来快速获得一个结果，然后再逐渐减小速率，获取最优的模型参数。

## 3. 偏差与方差
机器学习的误差来自通常来自两方面：
$$总误差 = 偏差 + 方差$$
当我们的模型表现不佳时，通常是出现两种问题，一种是 高偏差 问题，另一种是 高方差 问题。识别它们有助于选择正确的优化方式，所以我们先来看下 偏差 与 方差 的意义。 
- 偏差(Bias): 描述模型输出结果的期望与样本真实结果的差距；可以理解为我们用训练集数据训练出来的模型的准确性。
- 方差(Variance): 描述模型对于给定值的输出稳定性；可以理解为模型预测新数据的准确性，也就是模型的泛化能力。
#### 偏差方差分布图
<img src="./material/方差与偏差.jpg" width="600px" height="600px">
就像打靶一样，偏差描述了我们的射击总体是否偏离了我们的目标，而方差描述了射击准不准。

接下来这张图表示了训练过程中偏差和方差的变化：
#### 训练中方差偏差的变化
<img src="./material/训练中的方差偏差.jpg" width="700px" height="700px">
图中纵坐标为方差，横坐标为使用的数据维度，当唯独数据选取较低时，我们的训练集误差和交叉验证集误差都会很大，这时偏差方差都很大；当维度选择刚好时，训练集误差和交叉验证集误差都很小；当维度过大时会产生过拟合，虽然训练集误差很小，但交叉验证集误差会很大，这时的误差主要是偏差较大影响的。

 ## 4. 过拟合与欠拟合
 当理解了方差和偏差的内容后，再来学习过拟合与欠拟合就轻而易举了。指的是模型的三个拟合状态：
 * 欠拟合：欠拟合很好理解，就是模型学习不足，对样本的预测准确度很低，在我们训练集中的结果都很差；偏差方差都很大。
 * 拟合合适：拟合的刚好合适，无论在训练集还是测试集的表现都不错；偏差方差都很小；
 * 过拟合：模型学习过度，不仅学习了应有的规则，也学习了训练集数据特有的噪声，导致训练集效果很好，用到了测试集上效果就很差了；表现为偏差小但方差大。
接下来我们从学习曲线来看下这三种状态：
#### 合适拟合
<img src="./material/合适拟合.jpg" width="600px" height="600px">

#### 欠拟合
<img src="./material/欠拟合.jpg" width="600px" height="600px">

#### 过拟合
<img src="./material/过拟合.jpg" width="600px" height="600px">

那当我们遇到过拟合和欠拟合问题，如何解决呢？以下是几种解决方式
#### 欠拟合

首先欠拟合就是模型没有很好地捕捉到数据特征，不能够很好地拟合数据。

- 构造更多特征；
- 增加数据量，更好的学习规则；
- 减少正则化参数

#### 过拟合
过拟合就是模型学习过度，过于复杂，不仅学习了应有的规则，也学习了训练集数据特有的噪声。解决方式：

- 增加数据量，本质还是数据不足没有描绘全貌；
- 进行正则化处理，减少模型复杂度，正则化可以有效减少过拟合；
- PCA降维，减少维度，减少噪声信息；
- 早停，减少训练的轮数。

## 5. 正则化
这里依旧要提一下**奥卡姆剃刀定律**,奥卡姆的威廉是 14 世纪一位崇尚简单的修士和哲学家。他认为科学家应该优先采用更简单（而非更复杂）的公式或理论。奥卡姆剃刀定律在机器学习方面的运用如下：

> **机器学习模型越简单，良好的实证结果就越有可能不仅仅基于样本的特性。**

而我们上面提到过了，正则化是一种减少过拟合的方法，它的本质也就在于减少模型的复杂程度。
<img src="./material/泛化曲线.svg" width="700" height="700px">

上图显示的是某个模型的训练损失逐渐减少，但验证损失最终增加。即该泛化曲线显示该模型与训练集中的数据过拟合。原因就是之前我们一直单单只以损失最小作为目的：
$$minimize(Loss(Data|Model))$$

但现在我们不能单单只以损失最小为目标，而是以最小化损失和复杂度为目标，这称为结构风险最小化：

$$minimize(Loss(Data|Model) + complexity(Model))$$

现在，我们的训练优化算法是一个由两项内容组成的函数：一个是损失项，用于衡量模型与数据的拟合度，另一个是正则化项，用于衡量模型复杂度。
对于模型来说，越是复杂的模型，使用的特征越多，特征的参数也会越大，减少模型的复杂程度同样也来自这两方面：
* 减少特征的维度，换句话说使某些特征的参数为0；
* 减少参数的权重；

这两种方式对应这L1正则化和L2正则化的方法。

### L1正则（参数稀疏性惩罚）
范数是一种数学上衡量向量长度或大小的基本概念，所谓L1正则，是指用1-范数的形式来作为正则化项，衡量模型复杂度。
$$1-范数： ||x||1 = |x_1| + |x_2| + ...+|x_n|$$即曼哈顿距离。
所以损失就是：$$L(w) = \frac{1}{N}(\sum_{i=1}^n{f(x_i;w_i) - y_i}^2) + \lambda||w||_1$$
前面一部分代表了模型的预测的损失，后一部分代表了正则化惩罚。我们的模型越复杂，精度越高，前一部分的结果越小；同时参数$w$的值会越大，会导致正则项增大；参数越小，后面部分就会越小，但模型的精度却会下降。添加了正则项相当与对损失添加了惩罚，这样训练出来的模型会在尽可能准确地情况下保持较好的泛化能力。

### L2正则（权重衰减惩罚）
理解了L1正则，L2正则就比较好理解了，同样是使用2-范数的形式作为正则化项。
$$1-范数： ||x||2 = |x_1|^2 + |x_2|^2 + ...+|x_n|^2$$也就是欧几里何距离。
总损失依旧是：$$L(w) = \frac{1}{N}(\sum_{i=1}^n{f(x_i;w_i) - y_i}^2) + \lambda||w||_1$$
和L1正则的作用类似。
### 区别
接下来我们看下两种正则化会带来怎样的结果。
<img src="./material/两种正则化.png" width="500px" height="500px">
左边是L1正则的结果，假设我们有两个参数$β_1和β_2$,由于L1正则是曼哈顿距离，所以所有可能的解在二维空间为一个倾斜的正方形，而椭圆代表了我们损失函数的解，两个图形接触的地方，就是我们的最优解；我们看到L1正则下，最优解会总在X轴或Y轴上，此时$β_1或β_2$一个参数为0，也就是L1正则往往会产生稀疏的解。在我们的训练中，相当于减少了特征的维度。

右边是L2正则的结果，L2正则依据欧几里何距离，所有的解构成一个圆，两个图形接触的地方，一般不在坐标轴，此时$β_1或β_2$相对来说都是较小，因此L2正则下，解一般都比较小，因此L2是对参数权重做衰减惩罚。

当我们的特征维度比较大时，一般会使用L1正则多些，维度较少时可以尝试使用L2正则。

## 6. 决策树原理
### 决策树简介
决策树是最重要的机器学习算法之一，也是所有基于树的集成算法，如随机森林、GDBT、xgboost等算法的基础，理解决策树的原理，对我们后期学习继承算法非常有帮助。
决策树基本生成原理很简单：在每个节点上，决策树会尝试所有特征的所有可能值，将数据分为两部分，根据**特征划分依据**，找到将数据分割最好的那个点作为分割的节点，然后对分割的两份数据重复这个步骤，最终将样本分为不同的类别。
在这里根据特征划分依据的不同，将决策树分为三类：
* ID3：将信息增益作为划分依据
* C4.5:将信息增益率作为划分依据
* CART：将基尼系数作为划分依据

这里，我们就以CART决策树为例，来详细学习决策树的生成过程。

### 基尼系数
基尼系数代表了模型的不纯度，基尼系数越小，则不纯度越低，特征越好。

具体的，在分类问题中，假设有K个类别，第k个类别的概率为pk, 则基尼系数的表达式为：
$$Gini(p) = \sum\limits_{k=1}^{K}p_k(1-p_k) = 1- \sum\limits_{k=1}^{K}p_k^2$$
从这个公式可以看出，样本的纯度越高，基尼系数的值就越小；极限情况下样本只有一类种类别，那么p=1，基尼系数为0.

对于给定的样本D,假设有K个类别, 第k个类别的数量为Ck,则样本D的基尼系数表达式为：
$$Gini(D) = 1-\sum\limits_{k=1}^{K}(\frac{|C_k|}{|D|})^2$$
特别的，对于样本D,如果根据特征A的某个值a,把D分成D1和D2两部分，则在特征A的条件下，D的基尼系数表达式为：

$$Gini(D,A)=\frac{|D1|}{|D|}Gini(D1)+\frac{|D2|}{|D|}Gini(D2)$$
而我们就是根据这个公式，来确定划分特征与特征值的。每个节点，决策树会遍历所有的可能特征与可能值，找到基尼系数最小的点，作为这个节点最优划分点，以此循环，生成整颗决策树。
### 生成决策树
算法输入是训练集D，基尼系数的阈值，用来对基尼系数进行约束，基尼系数足够小就直接停止递归；样本个数阈值，样本个数进行约束，最少每类多少样本，停止递归的限制条件。
输出是决策树T。

我们的算法从根节点开始，用训练集递归的建立CART树。

1) 对于当前节点的数据集为D，如果样本个数小于阈值或者没有特征，则返回决策子树，当前节点停止递归。

2) 计算样本集D的基尼系数，如果基尼系数小于阈值，则返回决策树子树，当前节点停止递归。

3) 计算当前节点现有的各个特征的各个特征值对数据集D的基尼系数。

4) 在计算出来的各个特征的各个特征值对数据集D的基尼系数中，选择基尼系数最小的特征A和对应的特征值a。根据这个最优特征和最优特征值，把数据集划分成两部分D1和D2，同时建立当前节点的左右节点，做节点的数据集D为D1，右节点的数据集D为D2.

5) 对左右的子节点递归的调用1-4步，生成决策树。
### 其他
决策树原理中，还有以下几点简单说下：
#### 缺失值
CART的决策树能够自动对缺失进行处理，不需要我们提前处理缺失也能进行训练；划节点时，他会将缺失值同时划分进两个两部分数据，不过会根据各部分数据量给一个权重，然后接着计算基尼系数。
#### 连续值处理
CART和C4.5不仅能解决分类问题，还能做回归预测，原理是对连续变量离散化。
m个样本的连续特征A有m个，从小到大排列为a1,a2,...,am,则CART算法取相邻两样本值的平均数，一共取得m-1个划分点，对于这m-1个点，分别计算以该点作为二元分类点时的基尼系数。选择基尼系数最小的点作为该连续特征的二元离散分类点。之后以同样的方式建立决策树，将不同的回归值看作不同的类别就可以了。
#### 剪枝
从上面的决策树建立流程可以看出，决策树是一种贪婪算法，他每次都选择当前划分最优点，因此决策树是一种非常容易过拟合的算法。对于树模型，减少过拟合的方式就是**剪枝**，可以理解为线性模型的中的正则化。
剪枝有两种：**预剪枝**和**后剪枝**
##### 预剪枝
预剪枝很简单，对树的生成做限制条件，通过提前停止树的构建而对树剪枝，比如定义一个树的深度，前面定义的样本阈值等，不满足条件就提前停止生长。
##### 后剪枝
后剪枝分两部分：一是先生成完整的决策树，然后以各种方式去掉一部分节点，得到剪枝后的模型；二对所有剪枝后的模型用交叉验证的方式来找到最好的决策树。

## 7. bagging&boosting
bagging和boosting是两种模型融合的方式，也就是将多个模型模型组合起来一起使用，形成一个性能更强大的融合模型。即将弱分类器组合成强分类起的方法。
这里我们只简单介绍一下：
### bagging
以下是bagging的生成原理：

A）从原始样本集中抽取训练集。共进行k轮抽取，得到k个训练集。

B）每次使用一个训练集得到一个模型，k个训练集共得到k个模型。

C）对分类问题：将上步得到的k个模型采用投票的方式得到分类结果；对回归问题，计算上述模型的均值作为最后的结果。
<img src="./material/bagging.jpg" width="500px" height="500px">
简单理解，训练多个模型，结果取所有模型投票结果或平均结果。
例如随机森林、各种bagging算法都是利用bagging的算法。

### boosting
与bagging的并行训练相比，boosting是一种串行训练的方式：

A）提高那些在前一轮被弱分类器分错样例的权值，减小前一轮分对样例的权值，来使得分类器更加关注分类错误情况。

B）通过加权相加将弱分类器进行线性组合，错误率小的分类器分配较大的权值，错误率较大的分类器分配较小的权值。

C）得到最终的模型结果。
<img src="./material/boosting.jpg" width="500px" height="500px">
例如GBDT、Adaboost、xgboost都是基于boosting的算法。

## 8. 分类评价指标auc
### 混淆矩阵
对于肿瘤预测的问题，我们可以使用一个 2x2 混淆矩阵来总结我们的模型，该矩阵描述了所有可能出现的结果（共四种）：

<img src="./material/混淆矩阵.png" width="600px" height="600px">
真正例是指模型将正类别样本正确地预测为正类别。同样，真负例是指模型将负类别样本正确地预测为负类别。
假正例是指模型将负类别样本错误地预测为正类别，而假负例是指模型将正类别样本错误地预测为负类别。

**准确率**是是一个用于评估分类模型的指标。通俗来说，准确率是指我们的模型预测正确的结果所占的比例。
在这里就是：$$\text{Accuracy} = \frac{TP+TN}{TP+TN+FP+FN} = \frac{1+90}{1+90+1+8} = 0.91$$
在 91 个良性肿瘤中，该模型将 90 个正确识别为良性。这很好。不过，在 9 个恶性肿瘤中，该模型仅将 1 个正确识别为恶性。这是多么可怕的结果！9 个恶性肿瘤中有 8 个未被诊断出来！如果我根本不用模型，是个人我就说他是良性，那我也有91%的准确率，那我们的模型又有什么用呢。

当我们使用分类不平衡的数据集（比如正类别标签和负类别标签的数量之间存在明显差异）时，单单准确率一项并不能反映全面情况。

### 精确率和召回率
**精确率**指标是指：在被识别为正类别的样本中，确实为正类别的比例是多少？即：
$$\text{精确率} = \frac{TP}{TP+FP} = \frac{1}{1+1} = 0.5$$
**精确率**是指：在所有正类别样本中，被正确识别为正类别的比例是多少？
$$\text{召回率} = \frac{TP}{TP+FN} = \frac{1}{1+8} = 0.11$$
### roc
ROC 曲线（接收者操作特征曲线）是一种显示分类模型在所有分类阈值下的效果的图表。该曲线绘制了以下两个参数：

- 真正例率
- 假正例率

真正例率 (TPR) 是召回率的同义词，因此定义如下：
$$TPR = \frac{TP} {TP + FN}$$
是指正样本中，分类为正的概率。

假正例率 (FPR) 的定义如下：
$$FPR = \frac{FP} {FP + TN}$$
是指负样本中，分类为正概率。

**我们希望，真正例率越大约好，假正例率越小越好。**

我们知道，在我们的二分类模型输出时，实际是输出[0, 1]之间的概率值，我们一般分类时会使用0.5做为分割，才得到0和1两种类的结果。而ROC 曲线，就是用于绘制采用不同分类阈值时的 TPR 与 FPR。降低分类阈值(比如0.5变为0.3，0.3以上就认为为1类)会导致将更多样本归为正类别，从而增加假正例和真正例的个数。下图显示了一个典型的 ROC 曲线。
<img src="./material/roc.svg" width="300px" height="300px">
显然，ROC曲线的横纵坐标都在[0,1]之间，自然ROC曲线的面积不大于1。现在我们来分析几个特殊情况，从而更好地掌握ROC曲线的性质：

1. (0,0)：假正例率和真正例率都为0，即分类器全部预测成负样本
2. (0,1)：假正例率为0，真正例率为1，全部完美预测正确，happy
3. (1,0)：假正例率为1，真正例率为0，全部完美预测错误，悲剧
4. (1,1)：假正例率和真正例率都为1，即分类器全部预测成正样本

TPR＝FPR，斜对角线，预测为正样本的结果一半是对的，一半是错的，代表随机分类器的预测效果
于是，我们可以得到基本的结论：ROC曲线在斜对角线以下，则表示该分类器效果差于随机分类器，反之，效果好于随机分类器，当然，我们希望ROC曲线尽量除于斜对角线以上，也就是向左上角（0,1）凸。
### auc
ROC曲线一定程度上可以反映分类器的分类效果，但是不够直观，我们希望有这么一个指标，如果这个指标越大越好，越小越差，于是，就有了AUC。AUC实际上就是ROC曲线下的面积。AUC直观地反映了ROC曲线表达的分类能力。

- AUC ＝ 1，代表完美分类器
- 0.5 < AUC < 1，优于随机分类器
- 0 < AUC < 0.5，差于随机分类器

## 9. 总结&课堂任务
### 总结
<img src="./material/第四章总结.png" width="600px" height="600px">

### 课堂任务
* 对与课堂讲到的知识点，认真去分析理解
* 预习下节课内容